In [ ]:
import numpy as np
import os
import imageio
from multiprocessing import Pool
import cv2

from matplotlib.pyplot import imshow

from PIL import Image

from tqdm.notebook import trange, tqdm

outPath = "/home/ubuntuos/WeatherUNet/WeatherTiles/scripts/output/cropped/"
path = "/home/ubuntuos/WeatherUNet/WeatherTiles/scripts/output/conus/"

In [ ]:
def getUncroppedImages(X, Y):
    files= os.listdir(path)
    images = []
    for file in files:
        if file.endswith(".png") and not os.path.isfile(os.path.join(outPath, "(" + str(X) + ", " + str(Y) + ")/", file[:-4]+"_(" + str(X) + ", " + str(Y) + ").png")) :
            images.append(file)
    return(images)

def cropImages(image_path):
    #for image_path in tqdm(images, desc='Images'):
    input_path = os.path.join(path, image_path)
    try:
        image = Image.open(input_path)
        #for i in trange(9, desc='Y', leave = False):
        for i in range(9):
            #for j in trange(9, desc='X', leave = False):
            for j in range(9):
                box = (0+256*j, 0+256*i, 256+256*j, 256+256*i)
                cropped = image.crop(box)
                fullpath = os.path.join(outPath, "("+str(j)+", " + str(i) +")/", image_path[:-4]+"_("+str(j)+", " + str(i) +").png")            
                imageio.imwrite(fullpath, cropped)
    except:
        os.remove(input_path)
        
if __name__ == '__main__':
    images = getUncroppedImages(8, 8)
    with Pool() as p:
        list(tqdm(p.imap(cropImages, images), desc='Images', total=len(images)))

In [ ]:
def getCroppedImages(X=None, Y=None):
    files= []
    images = []
    if not X and not Y:
        for i in range(9):
            for j in range(9):
                files.extend(os.listdir(os.path.join(outPath, "(" + str(i) + ", " + str(j) + ")/")))
    else:
        files.extend(os.listdir(os.path.join(outPath, "(" + str(X) + ", " + str(Y) + ")/")))

    for file in files:
        if file.endswith(".png"):
            images.append(file)
    return(images)

def removeIncompleteImages(image_path):
    input_path = os.path.join(outPath + "(" + image_path[-6] + ", " + image_path[-9] + ")/", image_path)

    image = np.array(cv2.imread(input_path, cv2.IMREAD_GRAYSCALE))
    if (image.shape != (256, 256)):
        os.remove(input_path)
        print(input_path)

if __name__ == '__main__':
    images = getCroppedImages(3,3)
    with Pool() as p:
        list(tqdm(p.imap(removeIncompleteImages, images), desc='Images', total=len(images)))